In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-sin-normalized/test_layer_1.h5
/kaggle/input/forest-cover-type-sin-normalized/train_layer_2.h5
/kaggle/input/forest-cover-type-sin-normalized/dnn_elevation.h5
/kaggle/input/forest-cover-type-sin-normalized/__notebook__.ipynb
/kaggle/input/forest-cover-type-sin-normalized/train_layer_2_scaled.h5
/kaggle/input/forest-cover-type-sin-normalized/train_scaled.h5
/kaggle/input/forest-cover-type-sin-normalized/__output__.json
/kaggle/input/forest-cover-type-sin-normalized/train_sin_origin.h5
/kaggle/input/forest-cover-type-sin-normalized/test_sin_noramlized.h5
/kaggle/input/forest-cover-type-sin-normalized/test.h5
/kaggle/input/forest-cover-type-sin-normalized/test_layer_2_scaled.h5
/kaggle/input/forest-cover-type-sin-normalized/test_sin_origin.h5
/kaggle/input/forest-cover-type-sin-normalized/train_layer_1.h5
/kaggle/input/forest-cover-type-sin-normalized/train_layer_3.h5
/kaggle/input/forest-cover-type-sin-normalized/test_layer_2.h5
/kaggle/input/forest-cover-

In [2]:
from IPython.display import display

In [3]:
train_df = pd.read_hdf('/kaggle/input/forest-cover-type-sin-normalized/train_scaled.h5').set_index('Id')
test_df = pd.read_hdf('/kaggle/input/forest-cover-type-sin-normalized/test_scaled.h5').set_index('Id')

target_all_df = pd.read_csv('/kaggle/input/covertype-target/target.csv', index_col='Id')
y_test = target_all_df.loc[test_df.index]

display(train_df.head(2))
display(test_df.head(2))
# display(y_test.head(2))

,Elevation,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,...,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,Sin(Aspect),Sin(Slope),Cos(Aspect),Cos(Slope)
Id,,,,,,,,,,,,,,,,,,,,,
1,2.596,0.258,0.000,0.51,0.866667,0.909804,0.580392,6.279,1,0,...,0,0,0,0,0,5,0.777146,0.052336,0.629320,0.998630
2,2.590,0.212,-0.006,0.39,0.862745,0.921569,0.592157,6.225,1,0,...,0,0,0,0,0,5,0.829038,0.034899,0.559193,0.999391


,Elevation,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Sin(Aspect),Sin(Slope),Cos(Aspect),Cos(Slope)
Id,,,,,,,,,,,,,,,,,,,,,
15121,2.680,0.0,0.0,2.684,0.768627,0.839216,0.611765,6.645,1,0,...,0,0,0,0,0,0,-0.104528,0.241922,0.994522,0.970296
15122,2.683,0.0,0.0,2.654,0.788235,0.847059,0.596078,6.675,1,0,...,0,0,0,0,0,0,0.000000,0.224951,1.000000,0.974370


In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)

NameError: name 'X_train' is not defined

In [5]:
X_train = train_df.drop(columns=['Cover_Type'])
y_train = train_df.Cover_Type

X_test = test_df.copy()

In [6]:
y_train_en = pd.get_dummies(y_train)
y_test_en = pd.get_dummies(y_test['Cover_Type'])

display(y_train_en.head(2))
display(y_test_en.head(2))

,1,2,3,4,5,6,7
Id,,,,,,,
1,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0


,1,2,3,4,5,6,7
Id,,,,,,,
15121,1,0,0,0,0,0,0
15122,1,0,0,0,0,0,0


## Define DNN for Predicting CoverType

In [7]:
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, SGD

def get_model(input_dim, lr=5e-4):
    inputs = Input(shape=(input_dim,))
    
    y = Dense(32, activation='relu', name='layer_1')(inputs)
#     y = BatchNormalization()(y)
    y = Dense(16, activation='relu', name='layer_2')(y)
#     y = BatchNormalization()(y)
#     y = Dense(8, activation='relu', name='layer_3')(y)
    output = Dense(7, activation='softmax', name='layer_out')(y)
    
    model = Model(inputs=inputs,
                 outputs=output)
    
    model.compile(optimizer=Adam(lr=lr),
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])
    
    return model


model = get_model(input_dim=X_train.shape[1])


batch_size = 32
epochs = 50

model.fit(X_train, y_train_en,
         batch_size=batch_size,
         epochs=epochs,
         shuffle=True,
         )

Using TensorFlow backend.


Epoch 1/50
15120/15120 [==============================] - 1s 54us/step - loss: 1.5562 - categorical_accuracy: 0.4172
Epoch 2/50
15120/15120 [==============================] - 1s 43us/step - loss: 0.9914 - categorical_accuracy: 0.6077
Epoch 3/50
15120/15120 [==============================] - 1s 44us/step - loss: 0.8921 - categorical_accuracy: 0.6284
Epoch 4/50
15120/15120 [==============================] - 1s 43us/step - loss: 0.8521 - categorical_accuracy: 0.6456
Epoch 5/50
15120/15120 [==============================] - 1s 43us/step - loss: 0.8260 - categorical_accuracy: 0.6540
Epoch 6/50
15120/15120 [==============================] - 1s 43us/step - loss: 0.8057 - categorical_accuracy: 0.6615
Epoch 7/50
15120/15120 [==============================] - 1s 43us/step - loss: 0.7899 - categorical_accuracy: 0.6710
Epoch 8/50
15120/15120 [==============================] - 1s 43us/step - loss: 0.7761 - categorical_accuracy: 0.6761
Epoch 9/50
15120/15120 [==============================] - 1s 43u

In [8]:
filename ='dnn_cover_type.h5'
model.save(filename)

# Predict Features

In [9]:
from keras.models import load_model

model = load_model(filename)

# Predicting Intermediate Features

In [10]:
for layer in model.layers[1:-1]:
    name = layer.name
    
    model_tmp = Model(inputs=model.inputs,
                     outputs=model.get_layer(name).output)

    X_train_tmp = model_tmp.predict(X_train)
    
    df_tmp = pd.DataFrame(X_train_tmp, index=X_train.index)
    df_tmp.columns = ['{}({})'.format(name, i) for i in range(len(df_tmp.columns))]
    filename = 'train_{}.h5'.format(name)
    df_tmp.to_hdf(filename, 
                  mode='w',
                  key='train_{}'.format(name))
    
    X_test_tmp = model_tmp.predict(X_test)
    df_tmp = pd.DataFrame(X_test_tmp, index=X_test.index)
    df_tmp.columns = ['{}({})'.format(name, i) for i in range(len(df_tmp.columns))]
    df_tmp.to_csv(filename)
    filename = 'test_{}.h5'.format(name)
    df_tmp.to_hdf(filename, 
                  mode='w',
                  key='test_{}'.format(name))
    

# Read Features

In [11]:
!ls .
!pwd

__notebook__.ipynb  dnn_cover_type.h5  test_layer_2.h5	 train_layer_2.h5
__output__.json     test_layer_1.h5    train_layer_1.h5
/kaggle/working


### Concate Features and Dataset

In [12]:
model_layer = Model(inputs=model.inputs,
                   outputs=model.get_layer('layer_2').output)

X_train_layer = model_layer.predict(X_train)
X_test_layer = model_layer.predict(X_test)

In [13]:
df_tmp = pd.DataFrame(X_train_layer,
                      index=X_train.index)

df_tmp.columns = ['{}({})'.format(name, i) for i in range(len(df_tmp.columns))]

X_train_layer_df = df_tmp


df_tmp = pd.DataFrame(X_test_layer,
                      index=X_test.index)

df_tmp.columns = ['{}({})'.format(name, i) for i in range(len(df_tmp.columns))]

X_test_layer_df = df_tmp

In [14]:
X_train_layer_df.head()

,layer_2(0),layer_2(1),layer_2(2),layer_2(3),layer_2(4),layer_2(5),layer_2(6),layer_2(7),layer_2(8),layer_2(9),layer_2(10),layer_2(11),layer_2(12),layer_2(13),layer_2(14),layer_2(15)
Id,,,,,,,,,,,,,,,,
1,2.076868,6.468665,0.000000,0.876485,0.0,3.744210,3.011071,0.405310,1.192518,0.000000,2.408247,5.973473,0.695399,3.170933,4.548407,0.0
2,1.938043,6.485446,0.000000,0.858002,0.0,3.808466,2.977168,0.434467,1.107423,0.000000,2.449375,5.984871,0.743405,3.103426,4.548814,0.0
3,4.085976,6.942213,0.817130,0.000000,0.0,2.818314,3.219006,2.740005,1.365771,1.764798,1.241555,3.077788,0.000000,2.593797,3.761395,0.0
4,4.055928,6.016935,0.491508,0.000000,0.0,2.902722,3.623993,1.501600,0.500453,0.446460,2.526165,3.599282,0.065877,4.168538,3.060487,0.0
5,1.973613,6.371079,0.000000,0.952325,0.0,3.779940,2.908867,0.338290,1.095232,0.000000,2.466232,5.864185,0.770172,3.105002,4.528230,0.0


In [15]:
X_test_layer_df.head()

,layer_2(0),layer_2(1),layer_2(2),layer_2(3),layer_2(4),layer_2(5),layer_2(6),layer_2(7),layer_2(8),layer_2(9),layer_2(10),layer_2(11),layer_2(12),layer_2(13),layer_2(14),layer_2(15)
Id,,,,,,,,,,,,,,,,
15121,3.324801,6.573097,0.814593,0.628406,0.0,3.504552,3.142819,0.904242,0.313216,1.075240,2.147918,3.297236,0.220411,3.206395,3.609221,0.0
15122,3.285838,6.590958,0.790561,0.663993,0.0,3.550058,3.180875,0.952754,0.207743,1.082641,2.270645,3.301671,0.229666,3.196005,3.611517,0.0
15123,3.521916,6.642797,0.940826,0.572330,0.0,3.447650,3.150051,1.100880,0.082758,1.147120,2.378062,2.917908,0.274676,3.325835,3.445044,0.0
15124,3.506131,6.592950,0.969973,0.554341,0.0,3.453692,3.170528,1.098719,0.044741,1.106135,2.455196,2.953430,0.314175,3.334966,3.457066,0.0
15125,3.488988,6.545513,0.995808,0.533644,0.0,3.453975,3.183402,1.091971,0.025286,1.076132,2.496639,2.984506,0.339895,3.331754,3.468201,0.0


In [16]:
X_train_all = pd.merge(X_train.reset_index(),
                       X_train_layer_df.reset_index(),
                       on='Id').set_index('Id')

X_test_all = pd.merge(X_test.reset_index(),
                       X_test_layer_df.reset_index(),
                       on='Id').set_index('Id')

In [17]:
y_test.shape

(565892, 1)

In [18]:
X_test_all.shape

(565892, 72)

In [19]:
%%time
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score

seed = 47

# sample_weight = compute_sample_weight(class_weight=class_weight_start_1,
#                                         y=y_train)

clf = XGBClassifier(n_estimators=400,
                    learning_rate= 0.1, 
                    max_depth= 50,  
                    objective= 'multi:softmax',
                    random_state= seed,
                    verbosity=1,
                    n_jobs=-1)


clf.fit(X_train_all, 
        y_train.to_numpy()-1,
#        sample_weight,
       )

predictions = clf.predict(X_test_all)

score = accuracy_score(predictions, y_test.to_numpy()-1)


print('score={}'.format(score))

score=0.7714281170258636
CPU times: user 12min 42s, sys: 2.78 s, total: 12min 44s
Wall time: 3min 16s


In [20]:
from sklearn.metrics import accuracy_score
score = accuracy_score(predictions, y_test.to_numpy()-1)
print(score)

0.7714281170258636
